In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline


In [2]:
orders_df = pd.read_csv("orders.csv")

orders_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
orders_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [4]:
users_df = pd.read_json("users.json")

users_df.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
users_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [6]:
conn = sqlite3.connect(':memory:')

with open('restaurants.sql', 'r') as file:
    sql_script = file.read()

conn.executescript(sql_script)

restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)

restaurants_df.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
restaurants_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    object 
 2   cuisine          500 non-null    object 
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


In [8]:
final_df = orders_df.merge(users_df, on='user_id', how='left')
final_df = final_df.merge(restaurants_df, on='restaurant_id', how='left')

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [10]:
Left joins are used to ensure all transactional order records are retained even if user or restaurant details are missing.


SyntaxError: expected 'else' after 'if' expression (2394097695.py, line 1)

In [ ]:
# Convert order_date to datetime using explicit format
final_df['order_date'] = pd.to_datetime(
    final_df['order_date'],
    format='%d-%m-%Y'
)

# Extract time-based features
final_df['order_month'] = final_df['order_date'].dt.month
final_df['order_year'] = final_df['order_date'].dt.year
final_df['order_day'] = final_df['order_date'].dt.day_name()

# Weekend flag
final_df['is_weekend'] = final_df['order_day'].isin(['Saturday', 'Sunday'])



In [ ]:
final_df.head()


In [ ]:
monthly_orders = final_df.groupby('order_month')['order_id'].count()

monthly_orders.plot(kind='line', marker='o', title='Monthly Order Trend')
plt.xlabel('Month')
plt.ylabel('Number of Orders')
plt.show()


In [ ]:
Orders show variation across months, indicating seasonality in food ordering behavior.


In [ ]:
final_df.columns


In [ ]:
final_df = final_df.rename(columns={'total_amount': 'order_amount'})


In [ ]:
membership_summary = final_df.groupby('membership').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('order_amount', 'mean'),
    total_revenue=('order_amount', 'sum')
)

membership_summary


In [ ]:
city_revenue = final_df.groupby('city')['order_amount'].sum().sort_values(ascending=False)

city_revenue.plot(kind='bar', title='City-wise Revenue Distribution')
plt.xlabel('City')
plt.ylabel('Total Revenue')
plt.show()


In [ ]:
cuisine_orders = final_df.groupby('cuisine')['order_id'].count().sort_values(ascending=False)

cuisine_orders.plot(kind='bar', title='Orders by Cuisine')
plt.xlabel('Cuisine')
plt.ylabel('Number of Orders')
plt.show()


In [ ]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("final_food_delivery_dataset.csv saved successfully!")


In [ ]:
## Conclusion & Business Recommendations

- Gold members contribute higher average order value and revenue.
- Certain cities dominate order volume, indicating strong market potential.
- Cuisine preferences vary significantly across cities.
- Targeted promotions and membership upselling can improve revenue.

This analysis converts raw data into actionable business insights.


In [ ]:
# Food Delivery Data Analysis – Hackathon Submission

## Objective
To integrate transactional (CSV), user (JSON), and restaurant (SQL) datasets into a single analytical dataset and derive insights on order trends, user behavior, city-wise performance, membership impact, and revenue distribution.
